In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
all_sites = pd.read_csv('../data/all_sites.csv')

In [3]:
all_sites3 = all_sites[6200:9300]

In [5]:
# Select model
nlp_model = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = AutoTokenizer.from_pretrained(nlp_model)
model = AutoModelForSequenceClassification.from_pretrained(nlp_model)

# Classifier
classifier = pipeline(
        'sentiment-analysis', 
        model=model, 
        tokenizer=tokenizer)

KeyboardInterrupt: 

In [9]:
def sentiment_analysis_bert_base_multilingual_uncased(review):
    global_stars = []
    n = 1500
    
    review_splitted = [(review[i:i+n]) for i in range(0, len(review), n)]    
    global_stars = (classifier(review_splitted))
    
    return global_stars


In [10]:
def apply_sentiment_model(df):
    try:
        df['stars'] = df['text'].apply(lambda x: sentiment_analysis_bert_base_multilingual_uncased(x))
        
    except RuntimeError:
        print(df['game'])
        df['stars'] = df['text'].apply(lambda x: np.nan)
        
    finally:
        df.to_csv('../data/labeled_texts3.csv', index=False)
    
    return df

In [11]:
apply_sentiment_model(all_sites3)

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,site,url_link,author,game,company,genre,platform,text,score,stars
6200,Gamereactor,https://www.gamereactor.es/mordhau-analisis/?s...,Mike Holmes,Mordhau,Triternion,Acción,PC,Mordhau es un nuevo slash 'em up multijugador ...,8.0,"[{'label': '4 stars', 'score': 0.5731961727142..."
6201,Gamereactor,https://www.gamereactor.es/kirby-triple-deluxe...,Julián Plaza,Kirby: Triple Deluxe,Nintendo,Plataformas,Nintendo 3DS,Se veía venir. Ya comentamos en las impresione...,9.0,"[{'label': '4 stars', 'score': 0.5029962062835..."
6202,3D Juegos,https://www.3djuegos.com/juegos/analisis/4744/...,Álvaro Castellano,AC/DC Live: Rock Band - Análisis,None,None,PS3 X360 Wii PS2,La moda de juegos musicales destinados a banda...,6.9,"[{'label': '3 stars', 'score': 0.5163149833679..."
6203,Gamereactor,https://www.gamereactor.es/nova-3-near-orbit-v...,Lee West,N.O.V.A. 3 Near Orbit Vanguard Alliance,Gameloft,FPS,"Android, iOS","La verdad es que, de la emoción, de todo salió...",9.0,"[{'label': '3 stars', 'score': 0.4042036831378..."
6204,Gamereactor,https://www.gamereactor.es/terraria-para-xbox-...,Lorenzo Mosna,Terraria para Xbox 360 / PS3,Re-Logic,RPG,"PC, Xbox 360, XBLA, PS3, PSN, PS Vita, Androi...",Un par de años atrás comenzaba a explotar la l...,8.0,"[{'label': '4 stars', 'score': 0.5080600976943..."
...,...,...,...,...,...,...,...,...,...,...
9295,Vandal,https://vandal.elespanol.com/analisis/ps4/unti...,Jorge Cano,Until Dawn: Rush of Blood - Análisis,Sony,Realidad Virtual Shooter en primera persona Su...,PS4,"Desde su nacimiento, la realidad virtual siemp...",6.8,"[{'label': '4 stars', 'score': 0.4899812042713..."
9296,3D Juegos,https://www.3djuegos.com/juegos/analisis/36724...,Rodrigo Villanueva,112 Operator - Análisis,None,None,PC Mac,¿Cómo es un día en la vida de un teleoperador ...,7.0,"[{'label': '4 stars', 'score': 0.4735572636127..."
9297,Vandal,https://vandal.elespanol.com/analisis/ps4/stri...,Juan Rubio,Strike Suit Zero: Director's Cut - Análisis,Born Ready Games,Shoot'em up Simulador espacial,PS4 PC Xbox One Switch,"Como tantos otros géneros, los títulos de disp...",7.3,"[{'label': '4 stars', 'score': 0.4384359121322..."
9298,Vandal,https://vandal.elespanol.com/analisis/ps3/fist...,Daniel Escandell,Fist of the North Star: Ken’s Rage - Análisis,Tecmo Koei,Acción Musou,PS3 Xbox 360,Los aficionados al manga (tebeo japonés) tiene...,7.8,"[{'label': '5 stars', 'score': 0.4420377314090..."


In [12]:
def stars_mean_to_score(review):
    
    points = []

    for classification in review:

        #Mean of fragment
        grade = int(classification['label'].split(' ')[0])
        points.append(grade)  
    score = round(np.mean(points), 2)

    return(score)   

In [13]:
all_sites3['stars_mean'] = all_sites3['stars'].apply(lambda x: stars_mean_to_score(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
def score_labels(score):
    scorelabel = f"'{round(score / 2)} starts'"
    return scorelabel

In [15]:
all_sites3['score_label'] = all_sites3['score'].apply(lambda x: score_labels(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
def score_half(score):
    return score / 2

In [17]:
all_sites3['score_adj'] = all_sites3['score'].apply(lambda x: score_half(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
all_sites3.to_csv('../data/scored_texts3.csv', index=False)

In [19]:
all_sites3

,site,url_link,author,game,company,genre,platform,text,score,stars,stars_mean,score_label,score_adj
6200,Gamereactor,https://www.gamereactor.es/mordhau-analisis/?s...,Mike Holmes,Mordhau,Triternion,Acción,PC,Mordhau es un nuevo slash 'em up multijugador ...,8.0,"[{'label': '4 stars', 'score': 0.5731961727142...",4.00,'4 starts',4.00
6201,Gamereactor,https://www.gamereactor.es/kirby-triple-deluxe...,Julián Plaza,Kirby: Triple Deluxe,Nintendo,Plataformas,Nintendo 3DS,Se veía venir. Ya comentamos en las impresione...,9.0,"[{'label': '4 stars', 'score': 0.5029962062835...",4.00,'4 starts',4.50
6202,3D Juegos,https://www.3djuegos.com/juegos/analisis/4744/...,Álvaro Castellano,AC/DC Live: Rock Band - Análisis,None,None,PS3 X360 Wii PS2,La moda de juegos musicales destinados a banda...,6.9,"[{'label': '3 stars', 'score': 0.5163149833679...",3.50,'3 starts',3.45
6203,Gamereactor,https://www.gamereactor.es/nova-3-near-orbit-v...,Lee West,N.O.V.A. 3 Near Orbit Vanguard Alliance,Gameloft,FPS,"Android, iOS","La verdad es que, de la emoción, de todo salió...",9.0,"[{'label': '3 stars', 'score': 0.4042036831378...",3.80,'4 starts',4.50
6204,Gamereactor,https://www.gamereactor.es/terraria-para-xbox-...,Lorenzo Mosna,Terraria para Xbox 360 / PS3,Re-Logic,RPG,"PC, Xbox 360, XBLA, PS3, PSN, PS Vita, Androi...",Un par de años atrás comenzaba a explotar la l...,8.0,"[{'label': '4 stars', 'score': 0.5080600976943...",3.25,'4 starts',4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9295,Vandal,https://vandal.elespanol.com/analisis/ps4/unti...,Jorge Cano,Until Dawn: Rush of Blood - Análisis,Sony,Realidad Virtual Shooter en primera persona Su...,PS4,"Desde su nacimiento, la realidad virtual siemp...",6.8,"[{'label': '4 stars', 'score': 0.4899812042713...",3.60,'3 starts',3.40
9296,3D Juegos,https://www.3djuegos.com/juegos/analisis/36724...,Rodrigo Villanueva,112 Operator - Análisis,None,None,PC Mac,¿Cómo es un día en la vida de un teleoperador ...,7.0,"[{'label': '4 stars', 'score': 0.4735572636127...",3.43,'4 starts',3.50
9297,Vandal,https://vandal.elespanol.com/analisis/ps4/stri...,Juan Rubio,Strike Suit Zero: Director's Cut - Análisis,Born Ready Games,Shoot'em up Simulador espacial,PS4 PC Xbox One Switch,"Como tantos otros géneros, los títulos de disp...",7.3,"[{'label': '4 stars', 'score': 0.4384359121322...",3.50,'4 starts',3.65
9298,Vandal,https://vandal.elespanol.com/analisis/ps3/fist...,Daniel Escandell,Fist of the North Star: Ken’s Rage - Análisis,Tecmo Koei,Acción Musou,PS3 Xbox 360,Los aficionados al manga (tebeo japonés) tiene...,7.8,"[{'label': '5 stars', 'score': 0.4420377314090...",5.00,'4 starts',3.90
